# Fall Detection Model (TFLite Compatible)

This notebook trains a TensorFlow/Keras model to predict falls using the `HR_IMU_Fall_Detection_Dataset_Labeled.csv` dataset, using a subset of features (orientation and heart rate), and converts it to TFLite for ESP32 deployment.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 1. Load Data

In [ ]:
# Load the labeled dataset
df = pd.read_csv('HR_IMU_Fall_Detection_Dataset_Labeled.csv')

# Display first few rows
df.head()

## 2. Data Preprocessing
Removing features as requested:
1) ax: x axis of accelerometer signal (g)
2) ay: y axis of accelerometer signal (g)
3) az: z axis of accelerometer signal (g)
4) droll: angular velocity of gyroscope
5) dpitch: angular velocity of gyroscope
6) dyaw: angular velocity of gyroscope

In [ ]:
# Check for missing values
print("Missing values:\n", df.isnull().sum())

# Drop rows with missing values if any
df = df.dropna()

# Drop 'Scenario_Label' (descriptive) and 'Subject_ID' (if present)
if 'Scenario_Label' in df.columns:
    df = df.drop(columns=['Scenario_Label'])

if 'Subject_ID' in df.columns:
    df = df.drop(columns=['Subject_ID'])

# Remove the specified features
features_to_remove = ['ax', 'ay', 'az', 'droll', 'dpitch', 'dyaw']
df = df.drop(columns=features_to_remove, errors='ignore')

# Check data types
print("\nData Types:\n", df.dtypes)

In [ ]:
# Define Features (X) and Target (y)
X = df.drop(columns=['Fall'])
y = df['Fall']

# Handle categorical data (e.g., Sex) if needed
if X['Sex'].dtype == 'object':
    X = pd.get_dummies(X, columns=['Sex'], drop_first=True)

# Convert to float32 (better for TFLite/Microcontrollers)
X = X.astype('float32')
y = y.astype('float32')

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("Remaining columns:", X.columns.tolist())

## 3. Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

## 4. Model Training (TensorFlow/Keras)

In [ ]:
# Define a simple Sequential model suitable for ESP32
fall_detection_model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

fall_detection_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

fall_detection_model.summary()

In [ ]:
# Train the model
history = fall_detection_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32,
    epochs=20,
    verbose=1
)

## 5. Evaluation

In [ ]:
# Plot accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Make predictions
y_pred_prob = fall_detection_model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Classification Report
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Fall', 'Fall'], yticklabels=['No Fall', 'Fall'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## 6. Convert to TensorFlow Lite
Converting the Keras model to a TFLite model, optimized for microcontrollers (ESP32).

In [ ]:
# --- CONVERT TO TENSORFLOW LITE ---
converter = tf.lite.TFLiteConverter.from_keras_model(fall_detection_model)

# Apply optimizations (quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open('fall_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("\nConversion Successful!")
print("Model saved to 'fall_detection_model.tflite'")

In [ ]:
# Generate C array for ESP32
import binascii

def convert_to_c_array(bytes_data, var_name="fall_detection_model"):
    hex_str = binascii.hexlify(bytes_data).decode('utf-8')
    hex_list = [f'0x{hex_str[i:i+2]}' for i in range(0, len(hex_str), 2)]
    
    c_str = f"// Auto-generated model data\n"
    c_str += f"#include <cstdint>\n\n"
    c_str += f"alignas(16) const unsigned char {var_name}[] = {{\n"
    for i in range(0, len(hex_list), 12):
        c_str += "  " + ", ".join(hex_list[i:i+12]) + ",\n"
    c_str += "};\n\n"
    c_str += f"const unsigned int {var_name}_len = {len(hex_list)};\n"
    return c_str

# Write to header file
c_header = convert_to_c_array(tflite_model)
with open('model_data.h', 'w') as f:
    f.write(c_header)

print("Success! 'model_data.h' generated. You can include this in your ESP32 project.")